init jeu de donnees et affichage des informations, on notte une difference entre les colonnes Total_Proteins de chaque table

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# A compléter
df_h1 = pd.read_json('./liver.json')
df_h2 = pd.read_csv('./liver.csv')
df_h1.head()
df_h2.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,ALT,AST,Total_Proteins,Albumin,Albumin_and_Globulin_Ratio,Result
0,41 to 60 yo,Male,7.0,2.0,174.0,NaN,14,7800.0 mg/dL,4.2,NaN,2
1,41 to 60 yo,Male,6.0,2.0,245.0,22.0,24,7100.0 mg/dL,3.4,0.9,1
2,21 to 40 yo,Male,11.0,5.0,191.0,37.0,41,7700.0 mg/dL,4.3,NaN,2
3,21 to 40 yo,Male,24.0,10.0,340.0,25.0,21,8300.0 mg/dL,4.5,NaN,1
4,21 to 40 yo,Male,6.0,2.0,202.0,NaN,41,8000.0 mg/dL,3.9,0.9,1


combination donnees

avant de combiner les tables on fait l'harmonisation de la colonne Total_Proteins

In [16]:
def check_protein_format(protein_str):
    return isinstance(protein_str, str) and protein_str.strip().endswith('mg/dL')

def extract_protein_value(protein_str):
    if pd.isna(protein_str):
        return pd.NA
    if check_protein_format(protein_str):
        try:
            return float(protein_str.strip().replace('mg/dL','').strip())
        except:
            return pd.NA
    try:
        return float(protein_str/1000)
    except:
        return pd.NA

# Aplicar la función al dataframe (df_h2 ya debería existir)
df_h2['Total_Proteins'] = df_h2['Total_Proteins'].apply(extract_protein_value)
df_h2['Total_Proteins'].head()

0    7.8
1    7.1
2    7.7
3    8.3
4    8.0
Name: Total_Proteins, dtype: object

on fait l'harmonisation des unites de  Total_Bilirubin et Direct_Bilirubin

In [19]:
def harmonize_skin_thickness(df):
    df = df.copy()
    if 'Total_Bilirubin' in df.columns and 'Direct_Bilirubin' in df.columns:
        df['Total_Bilirubin'] = df['Total_Bilirubin'] / 10
        df['Direct_Bilirubin'] = df['Direct_Bilirubin'] / 10
    return df

df_h2 = harmonize_skin_thickness(df_h2)
df_h2[['Total_Bilirubin','Direct_Bilirubin']].head()

,Total_Bilirubin,Direct_Bilirubin
0,0.7,0.2
1,0.6,0.2
2,1.1,0.5
3,2.4,1.0
4,0.6,0.2


Albumin_and_Globulin_Ratio = Albumin/(Total_protein -Albumin) alors on verifie la coherence 

In [ ]:
def check_agr_consistency(df):
    df = df.copy()
    if 'Albumin' in df.columns and 'Total_protein' in df.columns and 'Albumin_and_Globulin_Ratio' in df.columns:
        agr_calc = df['Albumin'] / (df['Total_protein']-df['Albumin'])
        
        if df['Albumin_and_Globulin_Ratio'].isna():
            df['Albumin_and_Globulin_Ratio'] = agr_calc
            
        diff = (df['Albumin_and_Globulin_Ratio'] - agr_calc).abs()
        return diff == 0
    else:
        import pandas as pd
        return pd.Series([False]*len(df), index=df.index)


consistency = check_agr_consistency(df_combined)
consistency.value_counts()

apres revison on decide d'effacer les colonnes Albumin et Total_protein 

In [ ]:
df_combined = df_combined.drop(columns=['Albumin', 'Total_protein'], errors='ignore')
df_combined.head()

init de la table et les libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy import stats
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

df = df_combined
df.head()

Convertir la variable catégorielle Age en variable numérique à l'aide de `LabelEncoding`

In [ ]:
le = LabelEncoder()
df_label_encoded = df.copy()
df_label_encoded['Age'] = le.fit_transform(df_label_encoded['Age'])
df_label_encoded.head()

Et la variable catégorielle Gender en variable numérique à l'aide de `OneHotEncoding`

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Gender'], drop_first=False)
df_encoded.head()

Concaténé les 2 jeux de données (à la fin)

In [4]:
df_combined = pd.concat([df_h1,df_h2])
df_combined.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Proteins,Albumin,Albumin_and_Globulin_Ratio,Outcome,ALT,AST,Result
0,61 to 80 yo,Female,0.7,0.1,187.0,NaN,18.0,6.8,3.3,0.90,1.0,NaN,NaN,NaN
1,61 to 80 yo,Male,10.9,5.5,699.0,64.0,100.0,7.5,NaN,0.74,1.0,NaN,NaN,NaN
2,61 to 80 yo,Male,7.3,4.1,490.0,NaN,68.0,7.0,3.3,NaN,1.0,NaN,NaN,NaN
3,41 to 60 yo,Male,1.0,0.4,182.0,14.0,20.0,6.8,3.4,1.00,1.0,NaN,NaN,NaN
4,61 to 80 yo,Male,3.9,2.0,195.0,27.0,59.0,7.3,2.4,0.40,1.0,NaN,NaN,NaN
